In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

2023-05-15 18:44:09.024113: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-15 18:44:09.115828: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-15 18:44:09.148430: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# check gpu

tf.test.gpu_device_name()

2023-05-15 18:44:20.401189: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-15 18:44:21.664182: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-15 18:44:21.668883: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-15 18:44:21.669283: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been bu

'/device:GPU:0'

In [15]:

path = r"/wickkiey_git/InputVideos/2.mp4"


In [76]:
import cv2

def read_random_frame(video_path, resize_factor=4):
    # Open the video file
    video = cv2.VideoCapture(video_path)

    # Get the total number of frames in the video
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    # Generate a random frame number
    random_frame_number = int(np.random.randint(0, total_frames))

    # Set the frame position to the random frame number
    video.set(cv2.CAP_PROP_POS_FRAMES, random_frame_number)

    # Read the frame
    ret, frame = video.read()

    # Check if the frame was successfully read
    if not ret:
        raise ValueError("Error reading video frame")

    # Resize the frame to 4 times smaller
    resized_frame = cv2.resize(frame, (frame.shape[1] // resize_factor, frame.shape[0] // resize_factor))

    # Release the video capture object
    video.release()
    
    # convert frames to RGB
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)
    
    # return tensorflow tensors with extended dimension
    resized_frame, frame =  tf.expand_dims(resized_frame, axis=0), tf.expand_dims(frame, axis=0)

    return tf.cast(resized_frame,tf.float32), tf.cast(frame, tf.float32)



In [77]:
resized_frame, original_frame = read_random_frame(path)



In [78]:
resized_frame.shape, original_frame.shape

(TensorShape([1, 180, 320, 3]), TensorShape([1, 720, 1280, 3]))

In [79]:

initializer = tf.random_normal_initializer(0., 0.02)

upscale = tf.keras.Sequential([
    tf.keras.layers.Conv2DTranspose(64, 4, strides=2, padding='same', kernel_initializer=initializer, use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2DTranspose(64, 4, strides=2, padding='same', kernel_initializer=initializer, use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(3, (3, 3), padding='same', activation='sigmoid'),
])

In [80]:
upscale.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

In [ ]:
# conver model to gp

In [81]:
# fit a tensorflow generator 

def generator():
    while True:
        yield read_random_frame(path)

In [82]:
upscale.fit_generator(generator(), steps_per_epoch=100, epochs=10)

/tmp/ipykernel_32154/1875622962.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  upscale.fit_generator(generator(), steps_per_epoch=100, epochs=10)


Epoch 1/10
100/100 [==============================] - 8s 65ms/step - loss: 17630.1465 - accuracy: 0.4145
Epoch 2/10
100/100 [==============================] - 6s 62ms/step - loss: 19413.6738 - accuracy: 0.4079
Epoch 3/10
100/100 [==============================] - 6s 61ms/step - loss: 19188.8281 - accuracy: 0.3347
Epoch 4/10
100/100 [==============================] - 6s 61ms/step - loss: 16744.7441 - accuracy: 0.4322
Epoch 5/10
 91/100 [==========================>...] - ETA: 0s - loss: 17147.0742 - accuracy: 0.4220